# Integrasjon av diskrete verdier ved inndeling i celler

Noen ganger ønsker vi å integrere diskrete data for å finne et gjennomsnitt av data over et område. Forskjellen fra det vi har gjort tidligere er at vi ikke har en funksjon vi kan evaluere i vilkårlige nettverks-punkter. I stedet får vi oppgitt noen diskrete datapunkter og blir bedt om å beregne et gjennomsnitt over disse.

Dette betyr i praksis at vi må beregne en endelig Riemann-sum. Dette brukes som et dobbeltintegral. I motsetning til situasjonen tidligere kan vi ikke (uten problemer eller å undersøke problemet nærmere) gå til grensen slik vi ville gjort i funksjonstilfellet.
Likevel kan vi se dette som en forberedelse på nettbaserte metoder som vi senere skal bruke i kapitlet om partielle differensiallikninger.

Vi har sett at gjennomsnittsverdi for en funksjon $f(x,y)$ er gitt som

$$\overline{f} = \frac{1}{\text{arealet til }D} \iint_D f(x,y) dA$$

I tidligere notater har vi sett på hvordan vi kan diskretisere funksjoner ved hjelp av Numpys mesh funksjon. 
Problemet vi skal nå kort se på hvordan vi kan håndtere diskret data. For eksempel hvis $f(x,y)=k$ er konstant, så gjelder 

$$\overline{f} = \frac{1}{\text{arealet til }D} \iint_D \underbrace{f(x,y)}_{=k} dA = \frac{1}{\text{arealet til }D} \cdot k \cdot\text{arealet til }D =k$$

Mer generelt, hvis vi kan dele $D$ i områder $D=D_1\cup D_2 \cup \ldots \cup D_n, n\in \mathbb{N}$ slik at $f(x,y)=k_i$ er konstant på hver $D_i$ så gjelder
$$\iint_D f(x,y) dA = \iint_{D_1}\underbrace{f(x,y)}dA + \cdots + \iint_{D_n} f(x,y)dA =  k_1 \cdot \text{areal til } D_1 + \cdots k_n \cdot \text{areal til } D_n    $$
Dermed er for en funksjon som bare ta endelig mange verdier på $D$:
$$ \overline{f} = \frac{1}{\text{arealet til }D} \iint_D f(x,y) dA = \sum_i k_i \frac{\text{areal til }D_i}{\text{areal til }D}$$

In [ ]:
import numpy as np

## Eksempel: Klimadata over Norge

Et vanlig eksempel hvor vi ser på diskrete data er vær- og klimadata. Disse samles inn ved hjelp av målinger i noen steder i landet. Her ser vi på et eksempel nedbør kart fra ekstremvær "Hans" i Norge (kilde Wikipedia)

<img src="https://upload.wikimedia.org/wikipedia/commons/8/8f/Nedb%C3%B8r_mandag_7._august_2023.png" alt="Nedbør mandag 7. august 2023" width="800"/>


Kart viser gjennomsnittlig nedbør over Sør-Norge. Kartet viser Vi kan lure nå om gjennomsnittlig nedbør i hele Sør-Norge på 07.08.2023 ifølge ekstremvær Hans. For dette må vi lage et rutenett over kartet.
Det gjør vi slik: 
Plasser origo i det sørvestlige hjørnet av kartet slik at alle verdier kan betraktes som liggende i første kvadrant, og dermed er alle positive. Del deretter hele kartet inn i rektangler. Vi bruker har en oppdeling som ble lagd av satellitdata fra Copernikus Sentinel 2 satellit som vist nede. 

<img src="https://ec.europa.eu/eurostat/statistics-explained/images/c/cd/2018_NO_01_GG2023.jpg" alt="Sentinel 2 satellitdata, indeling Norge" width="800"/>

Cellene i bildet over er 100km x 100km stor. Vi ignorerer for nå at de ligger over hverandre ved rand (vi lagger en tilnærming uansett. (OBS: Verden er ikke en kvadratisk overflate så det er ikke forundrelig at det er ikke mulig å plassere kvadrater uten overlapp pent på en kart av Norge). For å lage en tilnærming kan vi bruke de 16 celler som ligger i den 4x4 blokk dens sør-vest- hjørnet ligger fra andre cellen til venstre av den røde cellen. Hvis vi estimerer fra nedbør kartet gjennomsnittlig nedbør i celler får vi en matrise $ij$-indeks i matrisen tilsvarer plass til cellene i den 4x4 gitter. 

In [ ]:
Kmatrise = np.array([[40, 40, 85, 65],[50, 75, 90, 20], [37, 52, 150, 60], [22,48,88,70]])
Kmatrise

Siden alle celler er like stor (her akkurat $\frac{1}{16}$), får vi fra formelen for middelverdien at 
$$\text{gjennomsnittlig regnmengde} = \frac{1}{16}\sum_{i,j} Kmatrise_{ij}

In [ ]:
gjen_regn = np.sum(Kmatrise)/16
print('Gjennomsnittlig regnmengde i Sør-Norge ved ekstremvær Hans var ', gjen_regn, 'mm')

## Regn på kysten

Under ekstremvær ble også regnmengden ved kysten målt. Hvis vi ser på celler som ligger vest til de celler vi har sett på finner vi celler ved kysten.
Regnmålinger for disse celler sammler vi i listen:

In [ ]:
regn_kysten = np.array([12,35,20,53])

Vi kan enkelt utvide gjennomsnittlig regnmengde ved å bruke de celler i den nye listen i tillegg til det vi har. 

$$\text{regn innlandet og kysten} = \frac{1}{20}\left(\underbrace{\sum_{i,j} \text{Kmatrise}_{ij}}_{\text{bidrag  innlandet}} + \underbrace{\sum_k \text{regn-kysten}_k}_{\text{kystens bidrag}}\right)$$ 


Merk at vi trenger å oppdatere tallet av celler vi deler med siden vi har nå $20$ celler.

In [ ]:
gjen_regn_med_kyst = (np.sum(Kmatrise)+np.sum(regn_ved_kysten))/20
print('Gjennomsnittlig regnmengde i Sør-Norge og kysten ved ekstremvær Hans var ', gjen_regn_med_kyst, 'mm')

De nye celler vi har tilføyet er ved kysten og inneholder dermed en god del hav hvor vi har ingen målinger siden målestasjoner ligger på land. 
Det er dermed ikke ønskelig å behandle disse celler som alle andre celler vi har allerede sett på siden de inneholder så mye hav.

### Ide: Reduser andel av nye celler i resultatet!

Når vi ser på kart er det maks 15% andel av land i enhver av cellene ved kysten. Dermed skulle bidrag til resultatet i disse cellene ikke overstige 15% av en full celle. 
Vi kan implementere det ved hjelp av å definere vekt til cellene:

Så hvis cellene ved kysten vektes bare som 15% av en full land celle kan vi multiplisere verdien av cellen med $0.15$ for å korrigiere verdien i summen.

OBS: Nå har vi 16 fulle celler og 4 celler som vektes hver med 15%. Dermed får vi ulike vekt for begge deler av summen. Se på følgende kode:

In [ ]:
gjen_regn_korrigert_kyst = (np.sum(Kmatrise)+.15*np.sum(regn_ved_kysten))/(16+4*0.15)
print('Gjennomsnittlig regnmengde i Sør-Norge og kysten ved ekstremvær Hans var ', gjen_regn_korrigert_kyst, 'mm')

## Sammenlikning av resultater
Tabellen nede viser sammenlikning av resultater.

In [ ]:
from tabulate import tabulate

# Data to display
headers = ["Beregning for ", "Gjennomsnittlig regn (mm)"]
data = [
    ["Innlandet uten kysten", gjen_regn],
    ["Innlandet oh kysten", gjen_regn_med_kyst],
    ["Innlandet og kyst korrigert", gjen_regn_korrigert_kyst]
]

# Display the table
print(tabulate(data, headers=headers, tablefmt="grid"))

## Hvordan kan vi implementere det mer konsepsjonelt?

VI har endret hvor mye en celle bidrar i siste beregning men har bare skillet to tilfeller:
1. Cellen er i innlandet
2. Cellen er ved kysten

Det kan godt være at vi ønsker å vekte cellene ulikt fordi di bidra ulik og vi har for hver cellen sin egen vekt som implementerer det.
I dette tilfelle blir det fort tungvint å implementere beregning på den måten vi har gjort:

$$\frac{1}{20}\left(\underbrace{\sum_{i,j} \text{Kmatrise}_{ij}}_{\text{bidrag  innlandet}} + \underbrace{\sum_k \text{regn-kysten}_k}_{\text{kystens bidrag}}\right)$$

Det vi egentlig vil er å beregne for hver celle

$$\text{verdi i cellen} \cdot \text{ vekt av cellen}$$

oppsummerer disse og dele med summen av vektene. Formelen blir der

$$\text{gjennomsnittlig regnmengde} = \frac{1}{\text{summen av vekter}}\sum_{celler} \text{verdi i cellen} \cdot \text{vekt av cellen}$$


In [ ]:
vekter_kysten = np.array([.15, .15, .15,.15])
vekter_innlandet = np.ones((4,4)) 

In [ ]:
gjen_regn = (np.sum(vekter_kysten * regn_kysten) + np.sum(vekter_innlandet * Kmatrise))/(np.sum(vekter_kysten)+np.sum(vekter_innlandet))
print(gjen_regn)

Resultatet er det samme som tidligere, men vi kna nå enkelt endre noen av de vektene for å endre hvordan en celle bidrar (uten at vi må tilpasse noe annet i koden).

For eksempel, hvis den siste kystcellen bidrar enda mindre (fordi landmasse i cellen er mindre enn 5%), så kan vi enkelt endre verdien til vekt for å beregne gjennomsnittsverdien på nytt.

In [ ]:
vekter_kysten[3]=.05
print(vekter_kysten)

In [ ]:
regn_ny= (np.sum(vekter_kysten * regn_kysten) + np.sum(vekter_innlandet * Kmatrise))/(np.sum(vekter_kysten)+np.sum(vekter_innlandet))
print('Gjennomsnittlig regnmengde ved ekstremvær Hans, nye vekt, var ', regn_ny, 'mm')

## TODO: Hvordan kan vi gå videre med resultatene

- Endre vekt for de andre celler for å unngå dobbeltelling pga overlapp. For dobbelt areal kan vi istedet bruke lineær interpolasjon mellom cellene